In [2]:
import logging
import numpy as np
import kubric as kb
from kubric.renderer.blender import Blender as KubricBlender
from kubric.simulator.pybullet import PyBullet as KubricSimulator

logging.basicConfig(level="INFO")  # < CRITICAL, ERROR, WARNING, INFO, DEBUG

# --- create scene and attach a renderer and simulator
scene = kb.Scene(resolution=(256, 256))
scene.frame_end = 48   # < numbers of frames to render
scene.frame_rate = 24  # < rendering framerate
scene.step_rate = 240  # < simulation framerate
renderer = KubricBlender(scene)
simulator = KubricSimulator(scene)

# --- populate the scene with objects, lights, cameras
scene += kb.Cube(name="floor", scale=(3, 3, 0.1), position=(0, 0, -0.1),
                 static=True)
scene += kb.DirectionalLight(name="sun", position=(-1, -0.5, 3),
                             look_at=(0, 0, 0), intensity=1.5)
scene.camera = kb.PerspectiveCamera(name="camera", position=(2, -0.5, 4),
                                    look_at=(0, 0, 0))

# --- generates spheres randomly within a spawn region
spawn_region = [[-1, -1, 0], [1, 1, 1]]
rng = np.random.default_rng()
for i in range(8):
  velocity = rng.uniform([-1, -1, 0], [1, 1, 0])
  material = kb.PrincipledBSDFMaterial(color=kb.random_hue_color(rng=rng))
  sphere = kb.Sphere(scale=0.1, velocity=velocity, material=material)
  scene += sphere
  kb.move_until_no_overlap(sphere, simulator, spawn_region=spawn_region)

/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
pybullet build time: Mar 26 2022 03:01:35


In [3]:
renderer.blender_scene.objects.values()

[bpy.data.objects['floor'],
 bpy.data.objects['sun'],
 bpy.data.objects['camera'],
 bpy.data.objects['Sphere'],
 bpy.data.objects['Sphere.001'],
 bpy.data.objects['Sphere.002'],
 bpy.data.objects['Sphere.003'],
 bpy.data.objects['Sphere.004'],
 bpy.data.objects['Sphere.005'],
 bpy.data.objects['Sphere.006'],
 bpy.data.objects['Sphere.007']]

In [10]:
cam = renderer.blender_scene.objects["camera"]
cam.matrix_world

Matrix(((0.2425355762243271, -0.8623489141464233, 0.4444444477558136, 2.0),
        (0.9701425433158875, 0.21558716893196106, -0.111111119389534, -0.5),
        (1.9165913300867032e-08, 0.4581228494644165, 0.8888888955116272, 4.0),
        (0.0, 0.0, 0.0, 1.0)))

In [4]:
import numpy as np


obj  = renderer.blender_scene.objects[5]
mesh = obj.to_mesh()


print(mesh.vertices)
print(mesh.polygons)
print(np.array(mesh.vertices[0].co), np.array(mesh.polygons[0].vertices))

<bpy_collection[2562], MeshVertices>
<bpy_collection[5120], MeshPolygons>
[ 0.  0. -1.] [ 0 27 26]


In [7]:
obj.matrix_world

Matrix(((-0.004497977439314127, -0.09926769137382507, -0.011211343109607697, 0.7945337891578674),
        (0.08669250458478928, 0.001697947271168232, -0.04981493577361107, 0.09796486049890518),
        (0.04964049905538559, -0.01196005567908287, 0.08598127216100693, 0.2595765292644501),
        (0.0, 0.0, 0.0, 1.0)))

In [90]:
print(obj)
print(obj.material_slots.items())

<bpy_struct, Object("Sphere.002") at 0x7a975c8>
[('PrincipledBSDFMaterial.002', bpy.data.objects['Sphere.002'].material_slots[0])]


In [92]:
material = mesh.materials[mesh.polygons[0].material_index]

shader_node_tree = material.node_tree
shader_node_tree.nodes.values()

print(shader_node_tree.nodes.get('Principled BSDF').color)
print(shader_node_tree.nodes.get('Material Output').color)


<Color (r=0.6080, g=0.6080, b=0.6080)>
<Color (r=0.6080, g=0.6080, b=0.6080)>


In [93]:
uv_layer = mesh.uv_layers.active
uv_layer

bpy.data.meshes['Icosphere.002'].uv_layers["UVMap"]

In [94]:
obj.data.vertex_colors.items()

[]

In [26]:
import bpy

# Set the object and face index you want to sample
# obj = bpy.context.object
face_index = 0  # Replace with the desired face index

# Ensure the object is in Object Mode
bpy.context.view_layer.objects.active = obj
bpy.ops.object.mode_set(mode='OBJECT')

# Get the mesh data
mesh = obj.data

# Check if vertex colors are present
if mesh.vertex_colors:
    # Access the vertex color layer
    vertex_color_layer = mesh.vertex_colors.active  # You can also specify the layer by name

    # Access the color for the specified face
    colors = vertex_color_layer.data[face_index].color

    print("Vertex Color of Face:", colors)
else:
    print("No vertex colors are defined on the mesh.")

No vertex colors are defined on the mesh.


In [56]:
import bpy

# Set the object and face index you want to sample
# obj = bpy.context.object
face_index = 0  # Replace with the desired face index
uv_coordinates = (0.5, 0.5)  # Replace with the desired UV coordinates (ranged from 0.0 to 1.0)

# Ensure the object is in Object Mode
bpy.context.view_layer.objects.active = obj
bpy.ops.object.mode_set(mode='OBJECT')

# Get the mesh data
mesh = obj.data

# Access the active material slot
material_index = 0  # Replace with the appropriate material slot index
material = obj.data.materials[material_index]

if material and material.use_nodes:
    shader_node_tree = material.node_tree
    shader_node = shader_node_tree.nodes.get('Principled BSDF')  # Assuming you're using the Principled BSDF shader
    if shader_node:
        # Get the texture coordinate node
        tex_coord_node = shader_node_tree.nodes.get('Texture Coordinate')
        print(shader_node_tree)
        print(shader_node_tree.nodes.values())
        print(shader_node)
        print(tex_coord_node)
        if tex_coord_node:
            # Set the active UV map for the face
            tex_coord_node.uv_map = "UVMap"  # Replace with the UV map name of your mesh

            # Access the image texture node
            image_texture_node = shader_node_tree.nodes.get('Image Texture')  # Replace with the name of your image texture node

            if image_texture_node and image_texture_node.image:
                # Get the color at the specified UV coordinates
                color = image_texture_node.image.sample(uv_coordinates)

                print("Color at UV Coordinates:", color)
            else:
                print("No valid image texture node found.")
        else:
            print("No Texture Coordinate node found.")
    else:
        print("No Principled BSDF shader node found in the material.")
else:
    print("No material applied to the object or material does not use nodes.")


<bpy_struct, ShaderNodeTree("Shader Nodetree") at 0x7471ba8>
[bpy.data.materials['PrincipledBSDFMaterial.001'].node_tree.nodes["Principled BSDF"], bpy.data.materials['PrincipledBSDFMaterial.001'].node_tree.nodes["Material Output"]]
<bpy_struct, ShaderNodeBsdfPrincipled("Principled BSDF") at 0x748edb8>
None
No Texture Coordinate node found.


In [34]:
import bpy

# Set the object and face index you want to sample
# obj = bpy.context.object
face_index = 0  # Replace with the desired face index
uv_coordinates = (0.5, 0.5)  # Replace with the desired UV coordinates (ranged from 0.0 to 1.0 in both dimensions)

# Ensure the object is in Object Mode
bpy.context.view_layer.objects.active = obj
bpy.ops.object.mode_set(mode='OBJECT')

# Get the mesh data
mesh = obj.data

# Check if there are UV coordinates defined
if mesh.uv_layers:
    # Access the UV map
    uv_map = mesh.uv_layers.active.data

    # Access the UV coordinates for the specified face
    face_uv = uv_map[face_index]

    # Use the specified UV coordinates
    face_uv.uv = uv_coordinates
    # face_uv.y = uv_coordinates[1]
    print(face_uv.uv.x)

    # Access the image texture
    image_texture = obj.data.textures[0]  # Replace with the appropriate texture slot index

    if image_texture and image_texture.image:
        # Get the color at the specified UV coordinates
        color = image_texture.image.sample(face_uv.uv.x, face_uv.uv.y)

        print("Color at UV Coordinates:", color)
    else:
        print("No valid image texture found.")
else:
    print("No UV coordinates defined on the mesh.")


0.5


AttributeError: 'Mesh' object has no attribute 'textures'

(array([ 0.,  0., -1.]), array([ 0, 27, 26]))

In [115]:
dir(mesh)

['__doc__',
 '__module__',
 '__slots__',
 'animation_data',
 'animation_data_clear',
 'animation_data_create',
 'asset_data',
 'attributes',
 'auto_smooth_angle',
 'auto_texspace',
 'bl_rna',
 'calc_loop_triangles',
 'calc_normals',
 'calc_normals_split',
 'calc_smooth_groups',
 'calc_tangents',
 'clear_geometry',
 'copy',
 'count_selected_items',
 'create_normals_split',
 'cycles',
 'edge_keys',
 'edges',
 'evaluated_get',
 'face_maps',
 'flip_normals',
 'free_normals_split',
 'free_tangents',
 'from_pydata',
 'has_custom_normals',
 'is_editmode',
 'is_embedded_data',
 'is_evaluated',
 'is_library_indirect',
 'library',
 'loop_triangles',
 'loops',
 'make_local',
 'materials',
 'name',
 'name_full',
 'normals_split_custom_set',
 'normals_split_custom_set_from_vertices',
 'original',
 'override_create',
 'override_library',
 'override_template_create',
 'polygon_layers_float',
 'polygon_layers_int',
 'polygon_layers_string',
 'polygons',
 'preview',
 'remesh_mode',
 'remesh_voxel_adapt